In [1]:
import pandas as pd
from pandas import DataFrame
import numpy as np
import datetime
from datetime import datetime, timedelta
import plotly.graph_objs as go
import matplotlib.pyplot as plt
import csv
import seaborn as sns
%matplotlib inline
sns.set()
from PIL import Image

# Dengue dataset:

The dataset is just the filtered confirmed cases, using spark, of the original dataset (see SINAN_extracao_dengue)

In [2]:
df = pd.read_csv('dengue_conf.csv',
                 low_memory = False, encoding="iso-8859-1")  

In [3]:
df.columns

Index(['tp_not', 'id_agravo', 'year_sin_pri', 'month_sin_pri', 'dt_notific',
       'sem_not', 'dt_sin_pri', 'sem_pri', 'classi_fin', 'criterio',
       'sg_uf_not', 'id_municip', 'cs_sexo', 'cs_gestant', 'cs_raca',
       'cs_escol_n', 'codmunres', 'dt_nasc', 'nu_idade_n', 'comuninf',
       'count'],
      dtype='object')

# Observe that the variable 'year_sin_pri' has several missprint. 

In [4]:
df.year_sin_pri.value_counts()

2015.0    1384175
2013.0    1175508
2010.0     864337
2011.0     596892
2016.0     474499
2014.0     474406
2012.0     405569
2017.0     166367
2018.0     144420
2009.0       7399
2001.0        434
1996.0        273
1994.0        264
1997.0        263
1993.0        255
1995.0        252
1989.0        248
1992.0        246
1982.0        245
1981.0        242
1991.0        238
1998.0        235
1983.0        233
1990.0        225
1978.0        225
1980.0        223
1986.0        222
1987.0        219
1988.0        216
1985.0        212
           ...   
1935.0         19
1937.0         17
1934.0         14
1931.0         13
1936.0         11
1932.0         11
1929.0          8
1933.0          7
1928.0          6
1930.0          5
1921.0          3
1925.0          3
1201.0          2
1923.0          2
1924.0          2
1926.0          2
1927.0          2
201.0           2
1922.0          1
1920.0          1
1918.0          1
1917.0          1
1915.0          1
1912.0          1
1911.0    

## We will create year and month from  variable 'dt_notific' in order to use in our work. (see dengue_case_file for the problematic relating to the variable tp extract the week)

In [5]:
df['Mycol'] =  df['dt_notific'].apply(pd.to_datetime)

In [6]:
df['year'] = pd.DatetimeIndex(df['Mycol']).year
df['month'] = pd.DatetimeIndex(df['Mycol']).month

In [7]:
df.year.value_counts()

2015    1385166
2013    1183153
2010     867811
2011     599758
2016     478547
2014     474849
2012     401390
2017     167453
2018     146306
Name: year, dtype: int64

# construct the dataset of dengue with the desired variables

In [8]:
data = df.filter(['codmunres','year','cs_sexo','nu_idade_n'])

## Process to clean age group and padronize following the age category given in population from IBGE

In [9]:
#data.nu_idade_n.unique()

In [10]:
vector =[]
for value in data.nu_idade_n.astype(str):
    if value == '1': value = 2001
    elif value == '2': value = 2001
    elif value == '3': value = 3001
    elif 4<= int(value) < 10: value = '400'+value
    elif len(value) == 2: value = '40' + value
    vector.extend([value])

In [11]:
#vector

In [12]:
data['nu_idade_n_cor'] = vector

In [13]:
data.columns

Index(['codmunres', 'year', 'cs_sexo', 'nu_idade_n', 'nu_idade_n_cor'], dtype='object')

In [ ]:
#data[data['nu_idade_n']==80]

In [ ]:
# Categorize age group
# 0,  101,  202,  303,  404,  505,  606,  707,  808,  909, 1010,
#       1111, 1212, 1313, 1414, 1515, 1616, 1717, 1818, 1919, 2024, 2529,
#       3034, 3539, 4044, 4549, 5054, 5559, 6064, 6569, 7074, 7579, 8099],

In [14]:
fx = []
for value in data.nu_idade_n_cor:
    if 1000 <= int(value) <4005: faixa = '4' 
    elif 4005 <=int(value)< 4010: faixa = '509'
    elif 4010 <=int(value)< 4015: faixa = '1014'
    elif 4015 <=int(value)< 4020: faixa = '1519'
    elif 4020 <=int(value)< 4025: faixa = '2024'
    elif 4025 <=int(value)< 4030: faixa = '2529'
    elif 4030 <=int(value)< 4035: faixa = '3034'
    elif 4035 <=int(value)< 4040: faixa = '3539' 
    elif 4040 <=int(value)< 4045: faixa = '4044'   
    elif 4045 <=int(value)< 4050: faixa = '4549'
    elif 4050 <=int(value)< 4055: faixa = '5054'
    elif 4055 <=int(value)< 4060: faixa = '5559'
    elif 4060 <=int(value)< 4065: faixa = '6064'
    elif 4065 <=int(value)< 4070: faixa = '6569'
    elif 4070 <=int(value)< 4075: faixa = '7074'
    elif 4075 <=int(value)< 4080: faixa = '7579'
    elif 4080 <=int(value)< 8100: faixa = '8099'
    fx.extend([faixa])             

In [15]:
data['faixa'] = fx
data

,codmunres,year,cs_sexo,nu_idade_n,nu_idade_n_cor,faixa
0,510340.0,2010,M,4014,4014,1014
1,350280.0,2010,F,4039,4039,3539
2,292740.0,2010,M,4036,4036,3539
3,110018.0,2010,F,4029,4029,2529
4,510704.0,2010,M,4029,4029,2529
5,510704.0,2010,M,4043,4043,4044
6,500370.0,2010,M,4042,4042,4044
7,130170.0,2010,F,4042,4042,4044
8,130170.0,2010,M,4052,4052,5054
9,510795.0,2010,F,4023,4023,2024


In [16]:
data.cs_sexo.value_counts()

F    3179812
M    2517543
I       6884
Name: cs_sexo, dtype: int64

In [17]:
data['count'] = 1

In [18]:
dados = data.groupby(['codmunres','year','cs_sexo','faixa'])['count'].sum().reset_index()

In [19]:
dados.cs_sexo = dados.cs_sexo.replace('F', 1)
dados.cs_sexo= dados.cs_sexo.replace('M', 2)

In [20]:
dados.head()

,codmunres,year,cs_sexo,faixa,count
0,110000.0,2015,1,4,1
1,110001.0,2010,1,1014,2
2,110001.0,2010,1,2024,1
3,110001.0,2010,1,3539,1
4,110001.0,2010,1,4549,1


In [21]:
dados.groupby(['year'])['count'].sum()

year
2010     867767
2011     599678
2012     401330
2013    1183011
2014     474775
2015    1384995
2016     478529
2017     167449
2018     146305
Name: count, dtype: int64

In [23]:
df.year.value_counts()

2015    1385166
2013    1183153
2010     867811
2011     599758
2016     478547
2014     474849
2012     401390
2017     167453
2018     146306
Name: year, dtype: int64

In [22]:
df.year_sin_pri.value_counts()

2015.0    1384175
2013.0    1175508
2010.0     864337
2011.0     596892
2016.0     474499
2014.0     474406
2012.0     405569
2017.0     166367
2018.0     144420
2009.0       7399
2001.0        434
1996.0        273
1994.0        264
1997.0        263
1993.0        255
1995.0        252
1989.0        248
1992.0        246
1982.0        245
1981.0        242
1991.0        238
1998.0        235
1983.0        233
1990.0        225
1978.0        225
1980.0        223
1986.0        222
1987.0        219
1988.0        216
1985.0        212
           ...   
1935.0         19
1937.0         17
1934.0         14
1931.0         13
1936.0         11
1932.0         11
1929.0          8
1933.0          7
1928.0          6
1930.0          5
1921.0          3
1925.0          3
1201.0          2
1923.0          2
1924.0          2
1926.0          2
1927.0          2
201.0           2
1922.0          1
1920.0          1
1918.0          1
1917.0          1
1915.0          1
1912.0          1
1911.0    

# Population

In [5]:
#pop = pd.read_csv('G:\Datalake\POPULACAO\csv\POPBR12.csv')                #'G:\Datalake\POPULACAO\pop_race_sex_age.csv')

In [29]:
pop = pd.read_csv('G:\Datalake\POPULACAO\estimativas_pop_sex_age_muni_cidacs\pop_2000_2017_SFA.csv')
pop = pop.drop(columns=['Unnamed: 0'])

In [30]:
pop.columns

Index(['MUNIC_RES', 'ANO', 'SEXO', 'FXETARIA', 'POPULACAO'], dtype='object')

In [31]:
pop.FXETARIA.unique()

array([   0,  101,  202,  303,  404,  505,  606,  707,  808,  909, 1010,
       1111, 1212, 1313, 1414, 1515, 1616, 1717, 1818, 1919, 2024, 2529,
       3034, 3539, 4044, 4549, 5054, 5559, 6064, 6569, 7074, 7579, 8099],
      dtype=int64)

In [32]:
pop = pop.replace({'FXETARIA': {0: 4, 101:4, 202:4,  303:4,  404:4,
                                505: 509, 606: 509,  707: 509,  808: 509,  909: 509,
                                1010: 1014, 1111: 1014, 1212: 1014, 1313: 1014, 1414: 1014,
                                1515: 1519, 1616: 1519, 1717: 1519, 1818: 1519, 1919: 1519}})

In [33]:
pop.head()

,MUNIC_RES,ANO,SEXO,FXETARIA,POPULACAO
0,110001,2000,1,4,118
1,110001,2000,1,4,110
2,110001,2000,1,4,127
3,110001,2000,1,4,149
4,110001,2000,1,4,140


# Creating tables to map the standard incidence ratio of dengue per year

## expected number of cases per year and cummulated

In [34]:
teste = dados

In [35]:
teste = teste[(2010 <= teste.year) & (teste.year <= 2018) & (teste.cs_sexo != 'I')]

In [36]:
teste.cs_sexo.unique()

array([1, 2], dtype=object)

In [37]:
teste.year.unique()

array([2015, 2010, 2011, 2012, 2013, 2014, 2017, 2018, 2016], dtype=int64)

In [51]:
teste['faixa'] = teste['faixa'].astype(int)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [52]:
teste.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 372306 entries, 0 to 374593
Data columns (total 5 columns):
codmunres    372306 non-null float64
year         372306 non-null int64
cs_sexo      372306 non-null object
faixa        372306 non-null int32
count        372306 non-null int64
dtypes: float64(1), int32(1), int64(2), object(1)
memory usage: 15.6+ MB


In [50]:
teste.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 372306 entries, 0 to 374593
Data columns (total 5 columns):
codmunres    372306 non-null float64
year         372306 non-null int64
cs_sexo      372306 non-null object
faixa        372306 non-null object
count        372306 non-null int64
dtypes: float64(1), int64(2), object(2)
memory usage: 17.0+ MB


In [53]:
sum_observed_gender_age_year = []
for year in [2010,2011,2012,2013,2014,2015,2016,2017,2018]:
    df_year = teste[teste.year == year]
    table_year = pd.pivot_table(df_year, values='count', index=['codmunres'], columns=['cs_sexo','faixa'])
    table_year=table_year.fillna(0)
    resulted_dataframe = table_year.sum().reset_index()
    sum_observed_gender_age_year.append(resulted_dataframe)

In [55]:
sum_observed_gender_age_year[0]

,cs_sexo,faixa,0
0,1,4,18141.0
1,1,509,25156.0
2,1,1014,38691.0
3,1,1519,46812.0
4,1,2024,49955.0
5,1,2529,49909.0
6,1,3034,46592.0
7,1,3539,41002.0
8,1,4044,37779.0
9,1,4549,34122.0


In [41]:
pop.head()

,MUNIC_RES,ANO,SEXO,FXETARIA,POPULACAO
0,110001,2000,1,4,118
1,110001,2000,1,4,110
2,110001,2000,1,4,127
3,110001,2000,1,4,149
4,110001,2000,1,4,140


# Population 

In [42]:
pop = pop.astype(int)

In [43]:
teste_pop = pop.groupby(['MUNIC_RES','ANO','SEXO','FXETARIA'])['POPULACAO'].sum().reset_index()
teste_pop.head()

,MUNIC_RES,ANO,SEXO,FXETARIA,POPULACAO
0,110001,2000,1,4,1496
1,110001,2000,1,509,1677
2,110001,2000,1,1014,1570
3,110001,2000,1,1519,1522
4,110001,2000,1,2024,1292


In [44]:
sum_pop_gender_age_year = []
for year in [2010,2011,2012,2013,2014,2015,2016,2017,2018]:
    df_year = teste_pop[teste_pop.ANO == year]
    table_year = pd.pivot_table(df_year, values='POPULACAO', index=['MUNIC_RES'], columns=['SEXO','FXETARIA'])
    table_year=table_year.fillna(0)
    resulted_dataframe = table_year.sum().reset_index()
    sum_pop_gender_age_year.append(resulted_dataframe)

In [45]:
sum_pop_gender_age_year[0].head()

,SEXO,FXETARIA,0
0,1,4,7016987
1,1,509,7624144
2,1,1014,8725413
3,1,1519,8558868
4,1,2024,8630227


In [31]:
#table_pop = pd.pivot_table(teste_pop, values='POPULACAO', index=['MUNIC_RES'], columns=['SEXO','FXETARIA'])
#table_pop

In [32]:
#sum_pop_gender_age_year = table_pop.sum().reset_index()

In [33]:
#sum_pop_gender_age_year.columns

In [34]:
#sum_pop_gender_age_year.rename(columns={0:'pop'}, 
#                 inplace=True)

In [35]:
#sum_pop_gender_age_year.head()

In [36]:
#sum_pop_gender_age_year['pop']

In [46]:
sum_pop_gender_age_year[0]

,SEXO,FXETARIA,0
0,1,4,7016987
1,1,509,7624144
2,1,1014,8725413
3,1,1519,8558868
4,1,2024,8630227
5,1,2529,8460995
6,1,3034,7717657
7,1,3539,6766665
8,1,4044,6320570
9,1,4549,5692013


# calculating r_ j (for each year)

In [86]:
#r_j = []
#for df in sum_observed_gender_age_year:
#    risco_relativo = df[0]/sum_pop_gender_age_year['pop']
#    r_j.append(risco_relativo)

In [56]:
r_j = []
for df1,df2 in zip(sum_observed_gender_age_year,sum_pop_gender_age_year):
    #df2.rename(columns={0:'pop'}, inplace=True)
    risco_relativo = df1[0]/df2[0]
    r_j.append(risco_relativo)

In [57]:
r_j[3]

0     0.002623
1     0.003232
2     0.005307
3     0.008151
4     0.008870
5     0.008180
6     0.008072
7     0.008110
8     0.008218
9     0.007950
10    0.007491
11    0.007041
12    0.006015
13    0.005487
14    0.004770
15    0.003775
16    0.002575
17    0.002613
18    0.003163
19    0.005261
20    0.007018
21    0.007156
22    0.006106
23    0.005608
24    0.005385
25    0.005344
26    0.004998
27    0.004482
28    0.004213
29    0.003835
30    0.003656
31    0.003270
32    0.002767
33    0.001925
Name: 0, dtype: float64

# calculating E_i for each municipality

In [58]:
table_pop = pd.pivot_table(teste_pop, values='POPULACAO', index=['MUNIC_RES'], columns=['SEXO','FXETARIA'])
table_pop

SEXO                   1                                               \
FXETARIA            4              509            1014           1519   
MUNIC_RES                                                               
110001       1215.555556    1302.222222    1328.277778    1353.944444   
110002       4295.444444    4377.611111    4564.666667    4488.055556   
110003        293.888889     315.055556     351.833333     344.500000   
110004       3455.055556    3532.166667    3831.888889    3970.111111   
110005        743.666667     787.555556     870.333333     863.111111   
110006        798.500000     841.222222     936.222222     960.555556   
110007        425.222222     470.444444     522.444444     500.500000   
110008        758.555556     754.777778     757.055556     689.888889   
110009       1360.333333    1349.888889    1442.388889    1419.777778   
110010       2292.222222    2323.388889    2311.166667    2184.444444   
110011       2505.388889    2638.444444    2810.055556    2851.888889   
110012       5187.611111    5351.666667    5598.555556    5816.333333   
110013       1532.944444    1595.055556    1660.888889    1627.000000   
110014        855.166667     956.500000     961.500000    1021.888889   
110015       1774.388889    1819.833333    1949.833333    1989.944444   
110018       1447.277778    1478.111111    1606.944444    1620.500000   
110020      20020.777778   19484.166667   20715.888889   20759.444444   
110025        988.333333    1106.333333    1202.055556    1246.333333   
110026        136.444444     152.222222     163.833333     162.500000   
110028       2270.222222    2388.388889    2499.833333    2517.333333   
110029        436.666667     466.666667     516.666667     524.111111   
110030       3303.722222    3371.722222    3546.555556    3554.722222   
110032       1296.111111    1362.888889    1363.444444    1363.277778   
110033       1102.388889    1186.611111    1224.111111    1167.666667   
110034        867.166667     908.500000     929.444444     943.277778   
110037        706.888889     777.000000     729.500000     726.111111   
110040        795.833333     890.444444     940.111111     907.444444   
110045       1869.888889    1923.777778    1919.055556    1932.055556   
110050        509.277778     552.444444     597.277778     613.500000   
110060        264.000000     285.666667     300.000000     291.888889   
...                  ...            ...            ...            ...   
522010        982.166667    1057.888889    1184.888889    1291.555556   
522015        181.611111     192.555556     212.388889     210.388889   
522020        937.833333     996.611111    1113.500000    1089.055556   
522026        137.666667     153.888889     173.666667     171.833333   
522028         64.166667      70.166667      73.333333      78.222222   
522040        605.833333     639.444444     677.611111     718.888889   
522045       4153.055556    3791.222222    3987.666667    3951.833333   
522050        268.111111     279.277778     300.000000     301.500000   
522060        765.944444     774.888889     858.000000     866.777778   
522068        323.666667     335.277778     370.111111     348.166667   
522070        114.722222     135.055556     161.777778     172.500000   
522100        103.111111     121.666667     135.388889     150.777778   
522108        165.666667     168.777778     184.888889     170.944444   
522119        301.555556     296.666667     304.555556     301.888889   
522130         97.944444     106.722222     123.722222     123.388889   
522140       4455.666667    4479.944444    4759.277778    4900.500000   
522145        135.277778     137.277778     158.555556     155.611111   
522150        166.055556     186.833333     204.222222     201.222222   
522155        197.111111     200.000000     201.222222     199.166667   
522157        116.000000     139.666667     147.000000     143.000000   
522160       1405.944444    1498.833333    1661.722222    

In [59]:
teste_pop.head()

,MUNIC_RES,ANO,SEXO,FXETARIA,POPULACAO
0,110001,2000,1,4,1496
1,110001,2000,1,509,1677
2,110001,2000,1,1014,1570
3,110001,2000,1,1519,1522
4,110001,2000,1,2024,1292


# 2010

In [60]:
filter_2010 = teste_pop[teste_pop.ANO == 2010]

In [ ]:
P_2010 = pd.pivot_table(filter_2010, values='POPULACAO', index=['MUNIC_RES'], columns=['SEXO','FXETARIA'])
P_2010;

In [62]:
E_2010 =[]
for i in range(len(P_2010)):
    E_i = sum(np.array(P_2010.iloc[i])*np.array(r_j[0]))
    E_2010.append([P_2010.index.values[i],E_i])

In [63]:
Expected_2010 = pd.DataFrame(E_2010,
                 columns=['codmunres', 'expected_2010'])

In [64]:
Expected_2010.head()

,codmunres,expected_2010
0,110001,112.564015
1,110002,416.513704
2,110003,28.952133
3,110004,361.927514
4,110005,78.174425


# 2011

In [66]:
filter_2011 = teste_pop[teste_pop.ANO == 2011]

In [ ]:
P_2011 = pd.pivot_table(filter_2011, values='POPULACAO', index=['MUNIC_RES'], columns=['SEXO','FXETARIA'])
P_2011;

In [68]:
E_2011 =[]
for i in range(len(P_2011)):
    E_i = sum(np.array(P_2011.iloc[i])*np.array(r_j[1]))
    E_2011.append([P_2011.index.values[i],E_i])

In [69]:
Expected_2011 = pd.DataFrame(E_2011,
                 columns=['codmunres', 'expected_2011'])

In [70]:
Expected_2011.head()

,codmunres,expected_2011
0,110001,76.978418
1,110002,292.024837
2,110003,19.547296
3,110004,249.939200
4,110005,53.354860


# 2012

In [71]:
filter_2012 = teste_pop[teste_pop.ANO == 2012]

In [ ]:
P_2012 = pd.pivot_table(filter_2012, values='POPULACAO', index=['MUNIC_RES'], columns=['SEXO','FXETARIA'])
P_2012;

In [73]:
E_2012 =[]
for i in range(len(P_2012)):
    E_i = sum(np.array(P_2012.iloc[i])*np.array(r_j[2]))
    E_2012.append([P_2012.index.values[i],E_i])

In [74]:
Expected_2012 = pd.DataFrame(E_2012,
                 columns=['codmunres', 'expected_2012'])

In [75]:
Expected_2012.head()

,codmunres,expected_2012
0,110001,50.685018
1,110002,196.030415
2,110003,12.753113
3,110004,167.052405
4,110005,35.209028


# 2013

In [76]:
filter_2013 = teste_pop[teste_pop.ANO == 2013]

In [ ]:
P_2013 = pd.pivot_table(filter_2013, values='POPULACAO', index=['MUNIC_RES'], columns=['SEXO','FXETARIA'])
P_2013;

In [78]:
E_2013 =[]
for i in range(len(P_2013)):
    E_i = sum(np.array(P_2013.iloc[i])*np.array(r_j[3]))
    E_2013.append([P_2013.index.values[i],E_i])

In [79]:
Expected_2013 = pd.DataFrame(E_2013,
                 columns=['codmunres', 'expected_2013'])

In [80]:
Expected_2013.head()

,codmunres,expected_2013
0,110001,132.040242
1,110002,545.690146
2,110003,35.336098
3,110004,478.998675
4,110005,97.143271


# 2014

In [84]:
filter_2014 = teste_pop[teste_pop.ANO == 2014]

In [85]:
P_2014 = pd.pivot_table(filter_2014, values='POPULACAO', index=['MUNIC_RES'], columns=['SEXO','FXETARIA'])
P_2014;

SEXO            1                                                         \
FXETARIA     4      509     1014    1519    2024    2529    3034    3539   
MUNIC_RES                                                                  
110001        880    892     979    1088     995     997     938     863   
110002       3747   3971    4589    4687    4376    4349    4114    3537   
110003        216    219     249     236     271     280     244     186   
110004       2934   2777    3549    3791    3581    3565    3579    3172   
110005        602    558     697     738     830     671     649     565   
110006        643    591     711     754     798     715     679     581   
110007        294    286     350     392     379     374     345     321   
110008        665    692     790     647     649     593     577     475   
110009       1198   1183    1388    1365    1370    1336    1445    1183   
110010       2067   2178    2263    2152    1966    1851    1743    1482   
110011       2118   2161    2455    2460    2347    2412    2520    2150   
110012       4718   4663    5196    5716    5656    5567    5513    4835   
110013       1379   1360    1646    1557    1373    1477    1384    1017   
110014        673    808     880    1026     975     735     700     750   
110015       1416   1356    1592    1657    1695    1549    1598    1297   
110018       1203   1195    1347    1445    1488    1498    1510    1291   
110020      19711  17913   21202   21532   20664   20644   19884   16663   
110025        657    811     911     941     943     900     909     728   
110026         85    101     141     121     171     101     110      89   
110028       2047   1942    2274    2449    2355    2558    2298    2112   
110029        281    266     325     376     354     355     367     324   
110030       3467   3304    3869    4005    3604    4237    3854    3231   
110032        854    902    1092    1154     979    1023     959     769   
110033       1076   1251    1402    1252    1142    1154    1137     942   
110034        557    633     692     790     767     626     704     598   
110037        483    592     627     638     624     475     450     425   
110040        680    824     990     909     633     770     759     722   
110045       1446   1653    1821    1809    1584    1782    1899    1649   
110050        331    326     445     469     400     360     398     350   
110060        188    252     301     260     200     231     251     220   
...           ...    ...     ...     ...     ...     ...     ...     ...   
522010        920    881    1173    1345    1144    1150    1212    1117   
522015        178    191     236     179     181     185     225     195   
522020        690    733     985     885     866     962     973     841   
522026        114    141     148     139     122     158     157     174   
522028         36     50      66      77      59     104      82      86   
522040        548    563     694     769     740     541     626     661   
522045       4349   3891    4473    4619    4258    4602    4579    4038   
522050        232    255     309     304     320     288     297     311   
522060        613    631     865     775     699     648     753     800   
522068        263    310     319     268     212     395     342     265   
522070         74     92     138     116     120     146     131     115   
522100         85     79     106     133     148     130     102     108   
522108        144    126     178     142     181     151     111      79   
522119        275    260     294     325     319     266     308     306   
522130         92     66     105     124      94     102     116     109   
522140       4188   3877    4992    5159    4559    5155    5543    5053   
522145        102    106     149     123     154     115     145     113   
522150        128    118     150     159     161     167     167     215   
522155        190    181     182     181     206

In [87]:
E_2014 =[]
for i in range(len(P_2014)):
    E_i = sum(np.array(P_2014.iloc[i])*np.array(r_j[4]))
    E_2014.append([P_2014.index.values[i],E_i])

In [88]:
Expected_2014 = pd.DataFrame(E_2014,
                 columns=['codmunres', 'expected_2014'])

In [89]:
Expected_2014.head()

,codmunres,expected_2014
0,110001,51.618441
1,110002,219.136817
2,110003,13.846170
3,110004,191.326784
4,110005,38.055219


# 2015

In [90]:
filter_2015 = teste_pop[teste_pop.ANO == 2015]

In [91]:
P_2015 = pd.pivot_table(filter_2015, values='POPULACAO', index=['MUNIC_RES'], columns=['SEXO','FXETARIA'])
P_2015;

In [92]:
E_2015 =[]
for i in range(len(P_2015)):
    E_i = sum(np.array(P_2015.iloc[i])*np.array(r_j[5]))
    E_2015.append([P_2015.index.values[i],E_i])

In [93]:
Expected_2015 = pd.DataFrame(E_2015,
                 columns=['codmunres', 'expected_2015'])

In [94]:
Expected_2015.head()

,codmunres,expected_2015
0,110001,147.478319
1,110002,655.686938
2,110003,38.039101
3,110004,551.716310
4,110005,110.586085


# 2016

In [95]:
filter_2016 = teste_pop[teste_pop.ANO == 2016]

In [96]:
P_2016 = pd.pivot_table(filter_2016, values='POPULACAO', index=['MUNIC_RES'], columns=['SEXO','FXETARIA'])
P_2016;

In [97]:
E_2016 =[]
for i in range(len(P_2016)):
    E_i = sum(np.array(P_2016.iloc[i])*np.array(r_j[6]))
    E_2016.append([P_2016.index.values[i],E_i])

In [98]:
Expected_2016 = pd.DataFrame(E_2016,
                 columns=['codmunres', 'expected_2016'])

In [99]:
Expected_2016.head()

,codmunres,expected_2016
0,110001,49.679935
1,110002,228.595864
2,110003,12.657027
3,110004,190.085039
4,110005,37.437470


# 2017

In [100]:
filter_2017 = teste_pop[teste_pop.ANO == 2017]

In [101]:
P_2017 = pd.pivot_table(filter_2017, values='POPULACAO', index=['MUNIC_RES'], columns=['SEXO','FXETARIA'])
P_2017;

In [102]:
E_2017 =[]
for i in range(len(P_2017)):
    E_i = sum(np.array(P_2017.iloc[i])*np.array(r_j[7]))
    E_2017.append([P_2017.index.values[i],E_i])

In [103]:
Expected_2017 = pd.DataFrame(E_2017,
                 columns=['codmunres', 'expected_2017'])

In [104]:
Expected_2017.head()

,codmunres,expected_2017
0,110001,17.025525
1,110002,82.097493
2,110003,4.328885
3,110004,66.763122
4,110005,12.912315


# 2018

In [105]:
filter_2018 = teste_pop[teste_pop.ANO == 2018]

In [106]:
P_2018 = pd.pivot_table(filter_2018, values='POPULACAO', index=['MUNIC_RES'], columns=['SEXO','FXETARIA'])
P_2018;

In [115]:
sum_observed_gender_age_year[8]

,cs_sexo,faixa,0
0,1,4,3657.0
1,1,509,4137.0
2,1,1014,5362.0
3,1,1519,7667.0
4,1,2024,8398.0
5,1,2529,7703.0
6,1,3034,7744.0
7,1,3539,7407.0
8,1,4044,6485.0
9,1,4549,5725.0


In [119]:
r_j[8] =  sum_observed_gender_age_year[8][0]/sum_pop_gender_age_year[0][0]

In [120]:
E_2018 =[]
for i in range(len(P_2010)):
    E_i = sum(np.array(P_2010.iloc[i])*np.array(r_j[8]))
    E_2018.append([P_2010.index.values[i],E_i])

In [121]:
Expected_2018 = pd.DataFrame(E_2018,
                 columns=['codmunres', 'expected_2018'])

In [122]:
Expected_2018.head()

,codmunres,expected_2018
0,110001,18.895068
1,110002,69.753228
2,110003,4.884340
3,110004,60.773789
4,110005,13.143709


# Observed number of cases per municipality and year

In [123]:
observed = df.groupby(['codmunres','year'])['count'].sum().reset_index()
observed.head()

,codmunres,year,count
0,110000.0,2015,1
1,110001.0,2010,13
2,110001.0,2011,1
3,110001.0,2012,2
4,110001.0,2013,159


In [124]:
observed = observed[(2010 <= observed.year) & (observed.year <= 2018)]

In [125]:
observed = pd.pivot_table(observed, values='count', index=['codmunres'], columns=['year'])

In [126]:
observed = observed.fillna(0) 
observed.head()

year,2010,2011,2012,2013,2014,2015,2016,2017,2018
codmunres,,,,,,,,,
110000.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
110001.0,13.0,1.0,2.0,159.0,5.0,10.0,0.0,10.0,3.0
110002.0,228.0,27.0,2.0,35.0,56.0,21.0,0.0,61.0,7.0
110003.0,61.0,13.0,42.0,35.0,3.0,92.0,0.0,5.0,3.0
110004.0,939.0,31.0,18.0,442.0,31.0,46.0,1.0,25.0,13.0


# calculate the standard incidence ratio

In [127]:
codigo_muni = pd.read_excel('G:\Datalake\POPULACAO\População\POP_python\DTB_BRASIL_MUNICIPIO.xls')

In [128]:
codigo_muni.head()

,UF,Nome_UF,Mesorregião Geográfica,Nome_Mesorregião,Microrregião Geográfica,Nome_Microrregião,Município,Código Município Completo,Nome_Município
0,11,Rondônia,2,Leste Rondoniense,6,Cacoal,15,1100015,Alta Floresta D'Oeste
1,11,Rondônia,2,Leste Rondoniense,6,Cacoal,379,1100379,Alto Alegre dos Parecis
2,11,Rondônia,2,Leste Rondoniense,3,Ariquemes,403,1100403,Alto Paraíso
3,11,Rondônia,2,Leste Rondoniense,5,Alvorada D'Oeste,346,1100346,Alvorada D'Oeste
4,11,Rondônia,2,Leste Rondoniense,3,Ariquemes,23,1100023,Ariquemes


In [129]:
codigo_muni.columns

Index(['UF', 'Nome_UF', 'Mesorregião Geográfica', 'Nome_Mesorregião',
       'Microrregião Geográfica', 'Nome_Microrregião', 'Município',
       'Código Município Completo', 'Nome_Município'],
      dtype='object')

In [130]:
codigo_muni.rename(columns={'Código Município Completo':'codmuni_full'}, 
                 inplace=True);

In [131]:
codigo_muni['codmuni'] = codigo_muni.codmuni_full.floordiv(10)

In [132]:
codigo_muni.head()

,UF,Nome_UF,Mesorregião Geográfica,Nome_Mesorregião,Microrregião Geográfica,Nome_Microrregião,Município,codmuni_full,Nome_Município,codmuni
0,11,Rondônia,2,Leste Rondoniense,6,Cacoal,15,1100015,Alta Floresta D'Oeste,110001
1,11,Rondônia,2,Leste Rondoniense,6,Cacoal,379,1100379,Alto Alegre dos Parecis,110037
2,11,Rondônia,2,Leste Rondoniense,3,Ariquemes,403,1100403,Alto Paraíso,110040
3,11,Rondônia,2,Leste Rondoniense,5,Alvorada D'Oeste,346,1100346,Alvorada D'Oeste,110034
4,11,Rondônia,2,Leste Rondoniense,3,Ariquemes,23,1100023,Ariquemes,110002


In [133]:
observed.columns

Int64Index([2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018], dtype='int64', name='year')

In [134]:
final = codigo_muni.set_index('codmuni').join(observed) 

In [135]:
final

,UF,Nome_UF,Mesorregião Geográfica,Nome_Mesorregião,Microrregião Geográfica,Nome_Microrregião,Município,codmuni_full,Nome_Município,2010,2011,2012,2013,2014,2015,2016,2017,2018
codmuni,,,,,,,,,,,,,,,,,,
110001,11,Rondônia,2,Leste Rondoniense,6,Cacoal,15,1100015,Alta Floresta D'Oeste,13.0,1.0,2.0,159.0,5.0,10.0,0.0,10.0,3.0
110037,11,Rondônia,2,Leste Rondoniense,6,Cacoal,379,1100379,Alto Alegre dos Parecis,27.0,0.0,5.0,113.0,4.0,9.0,0.0,6.0,2.0
110040,11,Rondônia,2,Leste Rondoniense,3,Ariquemes,403,1100403,Alto Paraíso,33.0,35.0,36.0,57.0,18.0,3.0,0.0,22.0,4.0
110034,11,Rondônia,2,Leste Rondoniense,5,Alvorada D'Oeste,346,1100346,Alvorada D'Oeste,70.0,276.0,84.0,515.0,13.0,34.0,0.0,6.0,1.0
110002,11,Rondônia,2,Leste Rondoniense,3,Ariquemes,23,1100023,Ariquemes,228.0,27.0,2.0,35.0,56.0,21.0,0.0,61.0,7.0
110045,11,Rondônia,1,Madeira-Guaporé,1,Porto Velho,452,1100452,Buritis,647.0,82.0,27.0,105.0,89.0,149.0,0.0,117.0,85.0
110003,11,Rondônia,2,Leste Rondoniense,8,Colorado do Oeste,31,1100031,Cabixi,61.0,13.0,42.0,35.0,3.0,92.0,0.0,5.0,3.0
110060,11,Rondônia,2,Leste Rondoniense,3,Ariquemes,601,1100601,Cacaulândia,3.0,0.0,1.0,0.0,7.0,12.0,0.0,0.0,0.0
110004,11,Rondônia,2,Leste Rondoniense,6,Cacoal,49,1100049,Cacoal,939.0,31.0,18.0,442.0,31.0,46.0,1.0,25.0,13.0


In [136]:
final = final.join(Expected_2018.set_index('codmunres')).join(Expected_2017.set_index('codmunres')).join(Expected_2016.set_index('codmunres')).join(Expected_2015.set_index('codmunres')).join(Expected_2014.set_index('codmunres')).join(Expected_2013.set_index('codmunres')).join(Expected_2012.set_index('codmunres')).join(Expected_2011.set_index('codmunres')).join(Expected_2010.set_index('codmunres'))

In [137]:
final.columns

Index([                     'UF',                 'Nome_UF',
        'Mesorregião Geográfica',        'Nome_Mesorregião',
       'Microrregião Geográfica',       'Nome_Microrregião',
                     'Município',            'codmuni_full',
                'Nome_Município',                      2010,
                            2011,                      2012,
                            2013,                      2014,
                            2015,                      2016,
                            2017,                      2018,
                 'expected_2018',           'expected_2017',
                 'expected_2016',           'expected_2015',
                 'expected_2014',           'expected_2013',
                 'expected_2012',           'expected_2011',
                 'expected_2010'],
      dtype='object')

In [138]:
final['SIR_2010'] = final[2010]/final['expected_2010']
final['SIR_2011'] = final[2011]/final['expected_2011']
final['SIR_2012'] = final[2012]/final['expected_2012']
final['SIR_2013'] = final[2013]/final['expected_2013']
final['SIR_2014'] = final[2014]/final['expected_2014']
final['SIR_2015'] = final[2015]/final['expected_2015']
final['SIR_2016'] = final[2016]/final['expected_2016']
final['SIR_2017'] = final[2017]/final['expected_2017']
final['SIR_2018'] = final[2018]/final['expected_2018']

In [139]:
final

,UF,Nome_UF,Mesorregião Geográfica,Nome_Mesorregião,Microrregião Geográfica,Nome_Microrregião,Município,codmuni_full,Nome_Município,2010,...,expected_2010,SIR_2010,SIR_2011,SIR_2012,SIR_2013,SIR_2014,SIR_2015,SIR_2016,SIR_2017,SIR_2018
codmuni,,,,,,,,,,,,,,,,,,,,,
110001,11,Rondônia,2,Leste Rondoniense,6,Cacoal,15,1100015,Alta Floresta D'Oeste,13.0,...,112.564015,0.115490,0.012991,0.039459,1.204178,0.096865,0.067807,0.000000,0.587353,0.158772
110037,11,Rondônia,2,Leste Rondoniense,6,Cacoal,379,1100379,Alto Alegre dos Parecis,27.0,...,58.885687,0.458515,0.000000,0.185351,1.646931,0.146815,0.108388,0.000000,0.594375,0.202454
110040,11,Rondônia,2,Leste Rondoniense,3,Ariquemes,403,1100403,Alto Paraíso,33.0,...,78.820556,0.418673,0.628958,0.965531,0.556536,0.434144,0.024753,0.000000,1.445491,0.302534
110034,11,Rondônia,2,Leste Rondoniense,5,Alvorada D'Oeste,346,1100346,Alvorada D'Oeste,70.0,...,77.379905,0.904628,5.246621,2.447620,5.548233,0.358872,0.327245,0.000000,0.503185,0.076962
110002,11,Rondônia,2,Leste Rondoniense,3,Ariquemes,23,1100023,Ariquemes,228.0,...,416.513704,0.547401,0.092458,0.010202,0.064139,0.255548,0.032027,0.000000,0.743019,0.100354
110045,11,Rondônia,1,Madeira-Guaporé,1,Porto Velho,452,1100452,Buritis,647.0,...,148.997613,4.342351,0.777942,0.382687,0.511251,1.071370,0.655986,0.000000,4.003843,3.408481
110003,11,Rondônia,2,Leste Rondoniense,8,Colorado do Oeste,31,1100031,Cabixi,61.0,...,28.952133,2.106926,0.665054,3.293314,0.990489,0.216666,2.418564,0.000000,1.155032,0.614208
110060,11,Rondônia,2,Leste Rondoniense,3,Ariquemes,601,1100601,Cacaulândia,3.0,...,26.288220,0.114120,0.000000,0.082760,0.000000,0.544093,0.325449,0.000000,0.000000,0.000000
110004,11,Rondônia,2,Leste Rondoniense,6,Cacoal,49,1100049,Cacoal,939.0,...,361.927514,2.594442,0.124030,0.107751,0.922758,0.162026,0.083376,0.005261,0.374458,0.213908


In [140]:
final.to_excel('standard_incidence_ratio_dengue.xls')

cs_sexo -> 'F'= 1
cs_sexo -> 'M'= 2

cs_raca -> 1- branca
           2- preta
           3- amarela
           4- parda
           5- indígena
           9 Ignorado
           
           
cor -> Branca- 1
        Preta- 2 
        Amarela- 3
        Parda- 4 
        Indigena- 5 
        Sem declaraçao- 9
                  
Sexo -> 'Mulheres' = 1
        'Homens' = 2 
       
Idade -> '0 a 4 anos': 4, 
         '5 a 9 anos': 509,
         '10 a 14 anos': 1014,
         '15 a 19 anos': 1519,
         '20 a 24 anos': 2024, 
         '25 a 29 anos': 2529, 
         '30 a 34 anos': 3034, 
         '35 a 39 anos': 3539,
         '40 a 44 anos': 4044, 
         '45 a 49 anos': 4549,
         '50 a 54 anos': 5054, 
         '55 a 59 anos': 5559,
         '60 a 64 anos': 6064, 
         '65 a 69 anos': 6569, 
         '70 a 74 anos': 7074, 
         '75 a 79 anos': 7579,
         '80 a 89 anos': 8099, 
         '90 a 99 anos': 8099 

In [6]:
df['cs_raca'] = df['cs_raca'].astype(float).astype(int)

In [7]:
key = df.agg('{0[codmunres]}{0[cs_sexo]}{0[cs_raca]}{0[faixa]}'.format, axis=1)
df['key'] = key

In [8]:
df.head()

,codmunres,year,cs_sexo,cs_raca,faixa,count,key
0,110000,2015,1,1,4,1,110000114
1,110001,2010,1,1,1014,2,110001111014
2,110001,2010,1,1,2024,1,110001112024
3,110001,2010,1,1,3539,1,110001113539
4,110001,2010,1,1,4549,1,110001114549


In [9]:
pop.head()

,Unnamed: 0,codmun,mame_muni,cor,Sexo,Idade,Total
0,0,1100015,Alta Floresta D'Oeste (RO),1,2,4,430
1,1,1100015,Alta Floresta D'Oeste (RO),1,2,509,393
2,2,1100015,Alta Floresta D'Oeste (RO),1,2,1014,484
3,3,1100015,Alta Floresta D'Oeste (RO),1,2,1519,524
4,4,1100015,Alta Floresta D'Oeste (RO),1,2,2024,462


In [10]:
pop['codmun'] = pop['codmun'].astype(str).str[:-1].astype(np.int64)
pop.head()

,Unnamed: 0,codmun,mame_muni,cor,Sexo,Idade,Total
0,0,110001,Alta Floresta D'Oeste (RO),1,2,4,430
1,1,110001,Alta Floresta D'Oeste (RO),1,2,509,393
2,2,110001,Alta Floresta D'Oeste (RO),1,2,1014,484
3,3,110001,Alta Floresta D'Oeste (RO),1,2,1519,524
4,4,110001,Alta Floresta D'Oeste (RO),1,2,2024,462


In [11]:
y = pop.agg('{0[codmun]}{0[Sexo]}{0[cor]}{0[Idade]}'.format, axis=1)

In [12]:
pop['key1'] = y

In [65]:
pop

,codmun,mame_muni,cor,Sexo,Idade,Total,key1
0,110001,Alta Floresta D'Oeste (RO),1,2,4,430,110001214
1,110001,Alta Floresta D'Oeste (RO),1,2,509,393,11000121509
2,110001,Alta Floresta D'Oeste (RO),1,2,1014,484,110001211014
3,110001,Alta Floresta D'Oeste (RO),1,2,1519,524,110001211519
4,110001,Alta Floresta D'Oeste (RO),1,2,2024,462,110001212024
5,110001,Alta Floresta D'Oeste (RO),1,2,2529,444,110001212529
6,110001,Alta Floresta D'Oeste (RO),1,2,3034,387,110001213034
7,110001,Alta Floresta D'Oeste (RO),1,2,3539,359,110001213539
8,110001,Alta Floresta D'Oeste (RO),1,2,4044,383,110001214044
9,110001,Alta Floresta D'Oeste (RO),1,2,4549,363,110001214549


In [14]:
df.year.unique()

array([2015, 2010, 2011, 2012, 2013, 2014, 2017, 2018, 2016, 2009, 2045,
       2083, 2084, 2067, 2077, 2066, 2095, 2090, 2089], dtype=int64)

In [15]:
pop = pop.drop(columns=['Unnamed: 0'])

In [16]:
data_year = []
lst = [2015, 2010, 2011, 2012, 2013, 2014, 2017, 2018, 2016, 2009]
for value in lst:
    data = df[df['year'] == value]
    data_year.append(data)

# Table of number of cases per gender, race, age-group and year in Brazil

In [ ]:
teste = df.groupby(['cs_sexo','cs_raca','faixa','year'])['count'].sum().reset_index()
teste.head()

In [ ]:
table1 = pd.pivot_table(teste, values='count', index=['cs_sexo','cs_raca','faixa'], columns=['year'])

In [ ]:
table1.to_excel('table1.xls')

# Table with population per gender, race, age-group and year in Brazil

In [88]:
teste2 = pop.groupby(['Sexo','cor','Idade'])['Total'].sum().reset_index()

In [92]:
teste2.to_excel('table2.xls')

In [49]:
def filter_data(dataframe,value_year,value_gender,value_race):
    data = dataframe[(dataframe.year == value_year) &(dataframe.cs_sexo == value_gender) & (dataframe.cs_raca==value_race)]
    return data

In [52]:
data_female_white_2010 = filter_data(df,2010,'1',1)

In [63]:
data_female_white_2010.head()

,codmunres,year,cs_sexo,cs_raca,faixa,count,key
1,110001,2010,1,1,1014,2,110001111014
2,110001,2010,1,1,2024,1,110001112024
3,110001,2010,1,1,3539,1,110001113539
4,110001,2010,1,1,4549,1,110001114549
5,110001,2010,1,1,5054,1,110001115054


In [56]:
data_female_white_2010.groupby(['year','faixa'])['count'].sum().reset_index()

,year,faixa,count
0,2010,4,5833
1,2010,509,7572
2,2010,1014,12969
3,2010,1519,16780
4,2010,2024,18018
5,2010,2529,18152
6,2010,3034,17085
7,2010,3539,15602
8,2010,4044,14859
9,2010,4549,13813


,cs_sexo,cs_raca,faixa,year,count
0,1,1,4,2009,15
1,1,1,4,2010,5833
2,1,1,4,2011,3612
3,1,1,4,2012,1872
4,1,1,4,2013,5131


In [17]:
result = []
for data in data_year:
    df1 =pop.set_index('key1').join(data.set_index('key')) 
    result.append(df1)

In [18]:
result[0]

,codmun,mame_muni,cor,Sexo,Idade,Total,codmunres,year,cs_sexo,cs_raca,faixa,count
110001111014,110001,Alta Floresta D'Oeste (RO),1,1,1014,427,NaN,NaN,NaN,NaN,NaN,NaN
110001111519,110001,Alta Floresta D'Oeste (RO),1,1,1519,503,NaN,NaN,NaN,NaN,NaN,NaN
110001112024,110001,Alta Floresta D'Oeste (RO),1,1,2024,445,NaN,NaN,NaN,NaN,NaN,NaN
110001112529,110001,Alta Floresta D'Oeste (RO),1,1,2529,444,110001.0,2015.0,1,1.0,2529.0,1.0
110001113034,110001,Alta Floresta D'Oeste (RO),1,1,3034,362,NaN,NaN,NaN,NaN,NaN,NaN
110001113539,110001,Alta Floresta D'Oeste (RO),1,1,3539,369,NaN,NaN,NaN,NaN,NaN,NaN
110001114,110001,Alta Floresta D'Oeste (RO),1,1,4,446,110001.0,2015.0,1,1.0,4.0,1.0
110001114044,110001,Alta Floresta D'Oeste (RO),1,1,4044,366,NaN,NaN,NaN,NaN,NaN,NaN
110001114549,110001,Alta Floresta D'Oeste (RO),1,1,4549,342,NaN,NaN,NaN,NaN,NaN,NaN
110001115054,110001,Alta Floresta D'Oeste (RO),1,1,5054,234,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
result[0].cs_raca.unique()
result[0].cor.unique()

array([1, 2, 3, 4, 5, 9], dtype=int64)

In [21]:
result[0][result[0].cs_raca ==1]

,codmun,mame_muni,cor,Sexo,Idade,Total,codmunres,year,cs_sexo,cs_raca,faixa,count
110001112529,110001,Alta Floresta D'Oeste (RO),1,1,2529,444,110001.0,2015.0,1,1.0,2529.0,1.0
110001114,110001,Alta Floresta D'Oeste (RO),1,1,4,446,110001.0,2015.0,1,1.0,4.0,1.0
110001212024,110001,Alta Floresta D'Oeste (RO),1,2,2024,462,110001.0,2015.0,2,1.0,2024.0,1.0
110001213034,110001,Alta Floresta D'Oeste (RO),1,2,3034,387,110001.0,2015.0,2,1.0,3034.0,1.0
110002111014,110002,Ariquemes (RO),1,1,1014,1455,110002.0,2015.0,1,1.0,1014.0,1.0
110002114549,110002,Ariquemes (RO),1,1,4549,1030,110002.0,2015.0,1,1.0,4549.0,1.0
110002116064,110002,Ariquemes (RO),1,1,6064,470,110002.0,2015.0,1,1.0,6064.0,1.0
110003111519,110003,Cabixi (RO),1,1,1519,126,110003.0,2015.0,1,1.0,1519.0,2.0
110003112024,110003,Cabixi (RO),1,1,2024,92,110003.0,2015.0,1,1.0,2024.0,2.0
110003112529,110003,Cabixi (RO),1,1,2529,113,110003.0,2015.0,1,1.0,2529.0,1.0


cs_sexo -> 'F'= 1
cs_sexo -> 'M'= 2

cs_raca -> 1- branca
           2- preta
           3- amarela
           4- parda
           5- indígena
           9 Ignorado
           
           
cor -> Branca- 1
        Preta- 2 
        Amarela- 3
        Parda- 4 
        Indigena- 5 
        Sem declaraçao- 9
                  
Sexo -> 'Mulheres' = 1
        'Homens' = 2 
       
Idade -> '0 a 4 anos': 4, 
         '5 a 9 anos': 509,
         '10 a 14 anos': 1014,
         '15 a 19 anos': 1519,
         '20 a 24 anos': 2024, 
         '25 a 29 anos': 2529, 
         '30 a 34 anos': 3034, 
         '35 a 39 anos': 3539,
         '40 a 44 anos': 4044, 
         '45 a 49 anos': 4549,
         '50 a 54 anos': 5054, 
         '55 a 59 anos': 5559,
         '60 a 64 anos': 6064, 
         '65 a 69 anos': 6569, 
         '70 a 74 anos': 7074, 
         '75 a 79 anos': 7579,
         '80 a 89 anos': 8099, 
         '90 a 99 anos': 8099 